In [ ]:
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '20px'
      break
    }
  }
  '''))
increase_font()
get_ipython().events.register('pre_run_cell', increase_font)
print("Hello Everyone")

<IPython.core.display.Javascript object>

Hello Everyone


# **Install Dependencies**

In [ ]:
# Download Files
!pip3 install gdown -U

# Neural Machine Translation Library
!pip3 install OpenNMT-py

!git clone https://github.com/moses-smt/mosesdecoder.git
!pip3 install indic_nlp_library
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!pip3 install -r indic_nlp_library/requirements.txt
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

!pip3 install https://github.com/rsennrich/subword-nmt/archive/master.zip
!pip3 install sacrebleu
!pip3 install ctranslate2
!pip3 install mosestokenizer

<IPython.core.display.Javascript object>

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=db8beb8c9441ca045567984bda60009f6711470e083797d6bc8d93742cd7fe37
  Stored in directory: /root/.cache/pip/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
     |████████████████████████████████| 216 kB 5.2 MB/s 
     |████████████████████████████████| 57 kB 5.1 MB/s 
     |████████████████████████████████| 73 kB 1.5 MB/s 
     |████████████████████████████████| 16.6 MB 263 kB/s 
     |████████████████████████████████| 1.2 MB 33.7 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0

In [ ]:
%%bash
export PYTHONPATH=$PYTHONPATH:/content/indic_nlp_library
export INDIC_RESOURCES_PATH=/content/indic_nlp_resources

echo $INDIC_RESOURCES_PATH
echo $PYTHONPATH

<IPython.core.display.Javascript object>

/content/indic_nlp_resources
/env/python:/content/indic_nlp_library


# **Download Data**

## **Download Samanantar Dataset**
Source: https://indicnlp.ai4bharat.org/samanantar/

In [ ]:
!wget https://storage.googleapis.com/samanantar-public/V0.2/data/en2indic/en-mr.zip
!unzip en-mr.zip

<IPython.core.display.Javascript object>

--2022-04-06 18:29:56--  https://storage.googleapis.com/samanantar-public/V0.2/data/en2indic/en-mr.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.152.128, 173.194.194.128, 173.194.198.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.152.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 769637789 (734M) [application/zip]
Saving to: ‘en-mr.zip’

en-mr.zip           100%[===================>] 733.98M   136MB/s    in 5.9s    

2022-04-06 18:30:02 (125 MB/s) - ‘en-mr.zip’ saved [769637789/769637789]

Archive:  en-mr.zip
   creating: en-mr/
 extracting: en-mr/train.mr          
 extracting: en-mr/train.en          


In [ ]:
!wc -l en-mr/*

<IPython.core.display.Javascript object>

  3288874 en-mr/train.en
  3288874 en-mr/train.mr
  6577748 total


## **Split Data into Train, Test and Validation sets**

In [ ]:
sourceValidLen = 1000
targetValidLen = 1000

sourceTestLen = 1000
targetTestLen = 1000

sourceTrainLen = 20000
targetTrainLen = 20000

<IPython.core.display.Javascript object>

In [ ]:
!mkdir data
sourceData = open("en-mr/train.en", 'r').readlines()
targetData = open("en-mr/train.mr", 'r').readlines()

sourceValid = sourceData[0:sourceValidLen]
targetValid = targetData[0:targetValidLen]

sourceTest = sourceData[sourceValidLen:sourceValidLen + sourceTestLen]
targetTest = targetData[targetValidLen:targetValidLen + targetTestLen]

sourceTrain = sourceData[sourceValidLen + sourceTestLen:sourceValidLen + sourceTestLen + sourceTrainLen]
targetTrain = targetData[targetValidLen + targetTestLen:targetValidLen + targetTestLen + targetTrainLen]

sourceTestFile = open("data/test.en", "w+")
for line in sourceTest:
  sourceTestFile.write(line.strip("\n") + "\n")
sourceTestFile.close()

targetTestFile = open("data/test.mr", "w+")
for line in targetTest:
  targetTestFile.write(line.strip("\n") + "\n")
targetTestFile.close()

sourceValidFile = open("data/valid.en", "w+")
for line in sourceValid:
  sourceValidFile.write(line.strip("\n") + "\n")
sourceValidFile.close()

targetValidFile = open("data/valid.mr", "w+")
for line in targetValid:
  targetValidFile.write(line.strip("\n") + "\n")
targetValidFile.close()

sourceTrainFile = open("data/train.en", "w+")
for line in sourceTrain:
  sourceTrainFile.write(line.strip("\n") + "\n")
sourceTrainFile.close()

targetTrainFile = open("data/train.mr", "w+")
for line in targetTrain:
  targetTrainFile.write(line.strip("\n") + "\n")
targetTrainFile.close()


<IPython.core.display.Javascript object>

# **Preprocess Data**

## **Lowercase English Text**

In [ ]:
!/content/mosesdecoder/scripts/tokenizer/lowercase.perl < data/train.en > data/train-low.en
!/content/mosesdecoder/scripts/tokenizer/lowercase.perl < data/test.en > data/test-low.en
!/content/mosesdecoder/scripts/tokenizer/lowercase.perl < data/valid.en > data/valid-low.en

<IPython.core.display.Javascript object>

In [ ]:
!head -10 data/train.en

<IPython.core.display.Javascript object>

Some are against it.
[ Footnote] Capernaum was considered to be Jesus home city in the district of Galilee. Mark 2: 1.
PM Narendra Modi and Amit Shah have already held rallies in the state.
childrens education
In many places there had been clashes.
He had joined BJP before the Rajya Sabha polls.
Uddhav Thackeray had visited Ayodhya before the Lok Sabha polls.
Further arrests are expected in the future.
Dont like it.
So, lets say we just put one electrode of lithium here another electrode of something some counter electrode here.


In [ ]:
!head -10 data/train-low.en

<IPython.core.display.Javascript object>

some are against it.
[ footnote] capernaum was considered to be jesus home city in the district of galilee. mark 2: 1.
pm narendra modi and amit shah have already held rallies in the state.
childrens education
in many places there had been clashes.
he had joined bjp before the rajya sabha polls.
uddhav thackeray had visited ayodhya before the lok sabha polls.
further arrests are expected in the future.
dont like it.
so, lets say we just put one electrode of lithium here another electrode of something some counter electrode here.


## **Tokenize English Test**

In [ ]:
!/content/mosesdecoder/scripts/tokenizer/tokenizer.perl < data/train-low.en > data/train-tok.en
!/content/mosesdecoder/scripts/tokenizer/tokenizer.perl < data/test-low.en > data/test-tok.en
!/content/mosesdecoder/scripts/tokenizer/tokenizer.perl < data/valid-low.en > data/valid-tok.en

<IPython.core.display.Javascript object>

Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1


In [ ]:
!head -10 data/train-tok.en

<IPython.core.display.Javascript object>

some are against it .
&#91; footnote &#93; capernaum was considered to be jesus home city in the district of galilee. mark 2 : 1 .
pm narendra modi and amit shah have already held rallies in the state .
childrens education
in many places there had been clashes .
he had joined bjp before the rajya sabha polls .
uddhav thackeray had visited ayodhya before the lok sabha polls .
further arrests are expected in the future .
dont like it .
so , lets say we just put one electrode of lithium here another electrode of something some counter electrode here .


## **Normalize Marathi Text**

In [ ]:
!python3 indic_nlp_library/indicnlp/normalize/indic_normalize.py data/train.mr data/train-norm.mr mr
!python3 indic_nlp_library/indicnlp/normalize/indic_normalize.py data/valid.mr data/valid-norm.mr mr
!python3 indic_nlp_library/indicnlp/normalize/indic_normalize.py data/test.mr data/test-norm.mr mr

<IPython.core.display.Javascript object>

In [ ]:
!head -10 data/train.mr 

<IPython.core.display.Javascript object>

तर काही जण विरोधही करत आहेत.
या माहितीचे परीक्षण करताना आपल्याला कळेल, की आजही येशूचा लोकांवर प्रभाव का पडत आहे. (w१० - E ०४ / ०१)
नरेंद्र मोदी आणि अमित शहा यांनी देशात अराजक माजवले आहे.
मुलाचे शिक्षण
अनेक ठिकाणी पडझडीच्या घटना घडल्या.
मात्र विधानसभा निवडणुकीआधी ते भाजपमध्ये गेले.
लोकसभा निवडणुकीच्या आधी पक्षप्रमुख उद्धव ठाकरे यांनी अयोध्येचा दौरा केला होता.
येत्या काळात आणखी काहींची धरपकड केली जाण्याची शक्यता आहे.
अजिबात आवडला नाही.
तर, आपण इथे लिथियमचे एक इलेक्ट्रोड (electrode) ठेवू या.


In [ ]:
!head -10 data/train-norm.mr

<IPython.core.display.Javascript object>

तर काही जण विरोधही करत आहेत.
या माहितीचे परीक्षण करताना आपल्याला कळेल, की आजही येशूचा लोकांवर प्रभाव का पडत आहे. (w१० - E ०४ / ०१)
नरेंद्र मोदी आणि अमित शहा यांनी देशात अराजक माजवले आहे.
मुलाचे शिक्षण
अनेक ठिकाणी पडझडीच्या घटना घडल्या.
मात्र विधानसभा निवडणुकीआधी ते भाजपमध्ये गेले.
लोकसभा निवडणुकीच्या आधी पक्षप्रमुख उद्धव ठाकरे यांनी अयोध्येचा दौरा केला होता.
येत्या काळात आणखी काहींची धरपकड केली जाण्याची शक्यता आहे.
अजिबात आवडला नाही.
तर, आपण इथे लिथियमचे एक इलेक्ट्रोड (electrode) ठेवू या.


## **Tokenize Marathi Text**

Note: Uncomment if the part after `if __name__ == '__main__':` in the file `indic_nlp_library/indicnlp/tokenize/indic_tokenize.py`

In [ ]:
!python3 indic_nlp_library/indicnlp/tokenize/indic_tokenize.py data/train-norm.mr data/train-tok.mr mr
!python3 indic_nlp_library/indicnlp/tokenize/indic_tokenize.py data/valid-norm.mr data/valid-tok.mr mr
!python3 indic_nlp_library/indicnlp/tokenize/indic_tokenize.py data/test-norm.mr data/test-tok.mr mr

<IPython.core.display.Javascript object>

In [ ]:
!head -10 data/train-tok.mr

<IPython.core.display.Javascript object>

तर काही जण विरोधही करत आहेत . 
या माहितीचे परीक्षण करताना आपल्याला कळेल , की आजही येशूचा लोकांवर प्रभाव का पडत आहे . ( w१० - E ०४ / ०१ ) 
नरेंद्र मोदी आणि अमित शहा यांनी देशात अराजक माजवले आहे . 
मुलाचे शिक्षण
अनेक ठिकाणी पडझडीच्या घटना घडल्या . 
मात्र विधानसभा निवडणुकीआधी ते भाजपमध्ये गेले . 
लोकसभा निवडणुकीच्या आधी पक्षप्रमुख उद्धव ठाकरे यांनी अयोध्येचा दौरा केला होता . 
येत्या काळात आणखी काहींची धरपकड केली जाण्याची शक्यता आहे . 
अजिबात आवडला नाही . 
तर , आपण इथे लिथियमचे एक इलेक्ट्रोड ( electrode ) ठेवू या . 


## **Apply Byte Pair Encoding (BPE)**

In [ ]:
!pip3 install https://github.com/rsennrich/subword-nmt/archive/master.zip

<IPython.core.display.Javascript object>

     - 133 kB 1.8 MB/s


In [ ]:
!mkdir codes
!mkdir data/bpe

!subword-nmt learn-bpe -s 8000 --num-workers 40 < data/train-tok.en > codes/codes.en

!subword-nmt apply-bpe --num-workers 40 -c codes/codes.en < data/train-tok.en > data/bpe/train-bpe.en
!subword-nmt apply-bpe --num-workers 40 -c codes/codes.en < data/test-tok.en > data/bpe/test-bpe.en
!subword-nmt apply-bpe --num-workers 40 -c codes/codes.en < data/valid-tok.en > data/bpe/valid-bpe.en


!subword-nmt learn-bpe --num-workers 40 -s 8000 < data/train-tok.mr > codes/codes.mr

!subword-nmt apply-bpe --num-workers 40 -c codes/codes.mr < data/train-tok.mr > data/bpe/train-bpe.mr
!subword-nmt apply-bpe --num-workers 40 -c codes/codes.mr < data/test-tok.mr > data/bpe/test-bpe.mr
!subword-nmt apply-bpe --num-workers 40 -c codes/codes.mr < data/valid-tok.mr > data/bpe/valid-bpe.mr


<IPython.core.display.Javascript object>

mkdir: cannot create directory ‘codes’: File exists
mkdir: cannot create directory ‘data/bpe’: File exists
100% 8000/8000 [00:10<00:00, 729.16it/s] 
100% 8000/8000 [00:18<00:00, 437.47it/s]


### Download BPE Data and Codes

In [ ]:
import gdown

!mkdir codes
!mkdir data/bpe

url = 'https://drive.google.com/drive/folders/1wGnMc4f-JPVj9mG8gX52FVxs3JSv7F1R?usp=sharing'
gdown.download_folder(url, quiet=True)

!cp en-mr-bpe/codes.en codes/codes.en
!cp en-mr-bpe/codes.mr codes/codes.mr
!cp en-mr-bpe/train-bpe.en data/bpe/train-bpe.en
!cp en-mr-bpe/train-bpe.mr data/bpe/train-bpe.mr
!cp en-mr-bpe/test-bpe.en data/bpe/test-bpe.en
!cp en-mr-bpe/test-bpe.mr data/bpe/test-bpe.mr
!cp en-mr-bpe/valid-bpe.en data/bpe/valid-bpe.en
!cp en-mr-bpe/valid-bpe.mr data/bpe/valid-bpe.mr

In [ ]:
!head -10 data/bpe/train-bpe.en

<IPython.core.display.Javascript object>

some are against it .
&#91; foot@@ note &#93; ca@@ per@@ na@@ um was considered to be jesus home city in the district of gal@@ ile@@ e. mark 2 : 1 .
pm narendra modi and amit shah have already held rallies in the state .
child@@ ren@@ s education
in many places there had been clashes .
he had joined bjp before the rajya sabha polls .
uddhav thackeray had visited ayodhya before the lok sabha polls .
further arre@@ sts are expected in the future .
dont like it .
so , lets say we just put one electro@@ de of li@@ thi@@ um here another electro@@ de of something some counter electro@@ de here .


In [ ]:
!head -10 data/bpe/train-bpe.mr

<IPython.core.display.Javascript object>

तर काही जण विरोध@@ ही करत आहेत . 
या माहि@@ तीचे परीक्षण करताना आपल्याला कळ@@ ेल , की आजही येशू@@ चा लोकां@@ वर प्रभाव का पडत आहे . ( w@@ १० - E ०@@ ४ / ०@@ १ ) 
नरेंद्र मोदी आणि अमित शहा यांनी देशात अ@@ राज@@ क माज@@ वले आहे . 
मुला@@ चे शिक्षण
अनेक ठिकाणी पड@@ झ@@ डीच्या घटना घडल्या . 
मात्र विधानसभा निवडणुकी@@ आधी ते भाजपमध्ये गेले . 
लोकसभा निवडणुकीच्या आधी पक्षप्रमुख उद्धव ठाकरे यांनी अयो@@ ध्ये@@ चा दौरा केला होता . 
येत्या काळात आणखी काही@@ ंची धर@@ प@@ कड केली जाण्याची शक्यता आहे . 
अजिबात आवड@@ ला नाही . 
तर , आपण इथे लि@@ थि@@ य@@ मचे एक इलेक्@@ ट्रो@@ ड ( e@@ le@@ c@@ tr@@ o@@ d@@ e ) ठेवू या . 


## **Create Vocabulary**

In [ ]:
!mkdir data/pre
vocab = """
save_data: data/pre

src_vocab: data/pre/vocab.en
tgt_vocab: data/pre/vocab.mr

overwrite: False

data:
    corpus_1:
        path_src: data/bpe/train-bpe.en
        path_tgt: data/bpe/train-bpe.mr
"""

createVocabYaml = open("vocab.yaml", 'w+')
createVocabYaml.write(vocab)
createVocabYaml.close() 

!onmt_build_vocab -config vocab.yaml -n_sample -1

<IPython.core.display.Javascript object>

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-04-06 18:36:49,237 INFO] Counter vocab from -1 samples.
[2022-04-06 18:36:49,237 INFO] n_sample=-1: Build vocab on full datasets.
[2022-04-06 18:36:49,250 INFO] corpus_1's transforms: TransformPipe()
[2022-04-06 18:36:50,357 INFO] Counters src:7927
[2022-04-06 18:36:50,357 INFO] Counters tgt:8322


# **Train NMT Model**

In [ ]:
!mkdir checkpoints
!mkdir tensorboard

train = """

save_data: data/pre

src_vocab: data/pre/vocab.mr
tgt_vocab: data/pre/vocab.en

overwrite: False

src_seq_length: 200
tgt_seq_length: 200

data:
    corpus_1:
      path_src: data/bpe/train-bpe.en
      path_tgt: data/bpe/train-bpe.mr
      transforms: [filtertoolong]
    valid:
      path_src: data/bpe/valid-bpe.en
      path_tgt: data/bpe/valid-bpe.mr
      transforms: [filtertoolong]

# Training
world_size: 1
gpu_ranks: [0]
master_port: 5001

# General opts
log_file: checkpoints/log.txt
save_model: checkpoints/model
tensorboard_log_dir: "tensorboard"
tensorboard: true
keep_checkpoint: 20
save_checkpoint_steps: 10000
average_decay: 0.0005
seed: 1234
report_every: 1
train_steps: 300000
valid_steps: 10000

# Batching
queue_size: 10000
bucket_size: 32768
pool_factor: 8192
batch_type: "tokens"
batch_size: 4096
valid_batch_size: 16
batch_size_multiple: 1
max_generator_batches: 0
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
warmup_steps: 8000
decay_method: "noam"
adam_beta1: 0.9
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0.0
param_init_glorot: "true"
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
enc_layers: 3
dec_layers: 3
heads: 4
rnn_size: 256
dec_rnn_size: 256
word_vec_size: 256
transformer_ff: 1024
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
share_decoder_embeddings: "true"
position_encoding: "true"

"""

trainYaml = open("train.yaml", 'w+')
trainYaml.write(train)
trainYaml.close() 

<IPython.core.display.Javascript object>

In [ ]:
! CUDA_VISIBLE_DEVICES=0 onmt_train -config train.yaml

<IPython.core.display.Javascript object>

[2021-12-19 07:45:30,834 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-12-19 07:45:30,834 INFO] Parsed 2 corpora from -data.
[2021-12-19 07:45:30,834 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-12-19 07:45:30,834 INFO] Loading vocab from text file...
[2021-12-19 07:45:30,834 INFO] Loading src vocabulary from data/pre/vocab.mr
[2021-12-19 07:45:30,856 INFO] Loaded src vocab has 8322 tokens.
[2021-12-19 07:45:30,860 INFO] Loading tgt vocabulary from data/pre/vocab.en
[2021-12-19 07:45:30,877 INFO] Loaded tgt vocab has 7927 tokens.
[2021-12-19 07:45:30,882 INFO] Building fields with vocab in counters...
[2021-12-19 07:45:30,891 INFO]  * tgt vocab size: 7931.
[2021-12-19 07:45:30,903 INFO]  * src vocab size: 8324.
[2021-12-19 07:45:30,904 INFO]  * src vocab size = 8324
[2021-12-19 07:45:30,904 INFO]  * tgt vocab size = 7931
[2021-12-19 07:45:30,907 INFO] Building model...
[2021-12-19 07:45:40,817 INFO] NMTModel(
  (en

# **Inference**

### Download Covid-19 Test Set

In [ ]:
url = 'https://drive.google.com/drive/folders/1hy80UglVd7dUA_osGbl50BF4NWbp2msd?usp=sharing'
gdown.download_folder(url, quiet=True)

<IPython.core.display.Javascript object>

['/content/covid-19-testset/test.en', '/content/covid-19-testset/test.mr']

### Download WAT 2021 testset

In [ ]:
url = 'https://drive.google.com/drive/folders/1v6B21vqxkvq86bAwQlzFFCDXpVklp7Zq?usp=sharing'
gdown.download_folder(url, quiet=True)

<IPython.core.display.Javascript object>

['/content/wat2021/test.en', '/content/wat2021/test.hi']

## **Translate**

Note: Uncomment if the part after `if __name__ == '__main__':` in the file `indic_nlp_library/indicnlp/tokenize/indic_tokenize.py`

In [ ]:
!mkdir test

!/content/mosesdecoder/scripts/tokenizer/lowercase.perl < covid-19-testset/test.en > covid-19-testset/test-low.en
!/content/mosesdecoder/scripts/tokenizer/tokenizer.perl < covid-19-testset/test-low.en > covid-19-testset/test-tok.en
!subword-nmt apply-bpe -c codes/codes.en < covid-19-testset/test-tok.en > covid-19-testset/test-bpe.en


#check the name of model files on dir "checkpoints/" before running.
! CUDA_VISIBLE_DEVICES=0 onmt_translate \
        -gpu 0 \
        -batch_size 4096 -batch_type tokens \
        -beam_size 5 \
        -model checkpoints/model.pt \
        -src covid-19-testset/test-bpe.en \
        -output test/test.mr \
        -replace_unk
    
! sed -r -i 's/(@@ )|(@@ ?$)//g' test/test.mr

! sed -r -i 's/ &amp;apos;//g' test/test.mr

!python3 indic_nlp_library/indicnlp/tokenize/indic_detokenize.py test/test.mr test/test-detok.mr mr


<IPython.core.display.Javascript object>

Tokenizer Version 1.1
Language: en
Number of threads: 1
[2021-12-19 10:13:34,830 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2021-12-19 10:14:22,644 INFO] PRED AVG SCORE: -0.6487, PRED PPL: 1.9131


# **Compute BLEU score**

In [ ]:
import sacrebleu

hypothesis = open("test/test-detok.mr", 'r').readlines()
reference = open("covid-19-testset/test.mr", 'r').readlines()

hypothesisSentences, referenceSentences = [], []

for i in range(len(hypothesis)):
    hypothesisSentence = hypothesis[i].strip("\n")
    referenceSentence = reference[i].strip("\n")
    hypothesisSentences.append(hypothesisSentence)
    referenceSentences.append(referenceSentence)

bleu = sacrebleu.corpus_bleu(hypothesisSentences, [referenceSentences])
print(bleu.score)

<IPython.core.display.Javascript object>

19.14265478509391


# **Deployment** 

In [ ]:
import ctranslate2

model = "checkpoints/model.pt"
converter = ctranslate2.converters.OpenNMTPyConverter(model)

!mkdir model_deploy 
output = "model_deploy"
converter.convert(output,
    force=True,
    # quantization="int8"
    )

<IPython.core.display.Javascript object>

'model_deploy'

In [ ]:
from mosestokenizer import MosesSentenceSplitter, MosesTokenizer

from indicnlp.tokenize import sentence_tokenize, indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

import codecs
from subword_nmt.apply_bpe import BPE


<IPython.core.display.Javascript object>

In [ ]:
## Tokenize
englishTokenizer = MosesTokenizer('en')
## BPE
englishBpeCodes = codecs.open("codes/codes.en", encoding='utf-8')
englishBpeEncoder = BPE(englishBpeCodes)
## Translator
translator = ctranslate2.Translator("model_deploy",
    # compute_type="int8"
    )

sourceSentence = input("Enter English Sentence: ")

# Lowercase
sourceSentence = sourceSentence.lower()

# Tokenize
sourceSentence = ' '.join(englishTokenizer(sourceSentence))

# Apply BPE
sourceSentence = englishBpeEncoder.process_line(sourceSentence).split(" ")

# Translate
targetSentence = translator.translate_batch([sourceSentence], beam_size=5, max_batch_size=16)

# Remove BPE
targetSentence = (' '.join(targetSentence[0].hypotheses[0]) + " ").replace("@@ ", "")

print("Marathi Sentence:", targetSentence)


<IPython.core.display.Javascript object>

Enter English Sentence: Rivers in India play an important role in the lives of people.
Marathi Sentence: भारतातील नद्या लोकांच्या जीवनात महत्त्वाची भूमिका बजावतात . 
